In [7]:
import os
import torch.backends.cudnn as cudnn
import yaml
from train import train
from utils import AttrDict
import pandas as pd

In [8]:
cudnn.benchmark = True
cudnn.deterministic = False

In [9]:
import torch

def check_cuda():
    print('\n======================================================\n')
    if torch.cuda.is_available():
        print("CUDA is available. You can use GPU acceleration.")
        print("CUDA version:", torch.version.cuda)
        print("PyTorch version:", torch.__version__)
        print("Device name:", torch.cuda.get_device_name(0))  # Get the name of the GPU device
        print("Number of CUDA devices:", torch.cuda.device_count())
    else:
        print("CUDA is not available. GPU acceleration is not supported.")
    print('\n======================================================\n')
    

check_cuda()



CUDA is available. You can use GPU acceleration.
CUDA version: 11.8
PyTorch version: 2.2.1+cu118
Device name: NVIDIA GeForce RTX 3060 Laptop GPU
Number of CUDA devices: 1




In [10]:
def get_config(file_path):
    with open(file_path, 'r', encoding="utf8") as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    if opt.lang_char == 'None':
        characters = ''
        for data in opt['select_data'].split('-'):
            csv_path = os.path.join(opt['train_data'], data, 'labels.csv')
            df = pd.read_csv(csv_path, sep='^([^,]+),', engine='python', usecols=['filename', 'words'], keep_default_na=False)
            all_char = ''.join(df['words'])
            characters += ''.join(set(all_char))
        characters = sorted(set(characters))
        opt.character= ''.join(characters)
    else:
        opt.character = opt.number + opt.symbol + opt.lang_char
    os.makedirs(f'./saved_models/{opt.experiment_name}', exist_ok=True)
    return opt

In [11]:
opt = get_config("config_files/en_filtered_config.yaml")
train(opt, amp=False)

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: all_data
opt.select_data: ['kh_sample']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    all_data	 dataset: kh_sample
all_data/kh_sample
sub-directory:	/kh_sample	 num samples: 70
num total samples of kh_sample: 70 x 1.0 (total_data_usage_ratio) = 70
num samples of kh_sample per batch: 4 x 1.0 (batch_ratio) = 4


--------------------------------------------------------------------------------
Total_batch_size: 4 = 4
--------------------------------------------------------------------------------
dataset_root:    all_data/kh_val	 dataset: /
all_data/kh_val/
sub-directory:	/.	 num samples: 70
--------------------------------------------------------------------------------
No Transformation module specified
model input parameters 64 600 20 1 256 256 162 4 None VGG BiLSTM CTC
Model:
DataParallel(
  (module): Model(
    (FeatureExtraction): VGG_FeatureExtractor(
      (ConvNet): Sequential(
        (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): ReLU(inplace=True)
        (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (6): Conv2d

SystemExit: 

/home/acleda/miniconda3/envs/easyocr-kh/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
